In [43]:
import pandas as pd
import numpy as np
import math
import random
import heapq

# Inputs

In [2]:
inCsv = pd.read_excel("../data/InputData.xlsx")

inRooms = {
    "Armes 200": 100,
    "Armes 111": 35,
    "Armes 204": 85,
    "Armes 201": 52,
    "Armes 205": 52,
    "Armes 208": 85,
}

deferralRate = 0.1

examsPerDay = 3
slotNames = {
    0: "Morning",
    1: "Afternoon",
    2: "Evening",
    3: "Night"
}

# Helper Functions

In [3]:
def index_to_timeslot(slotIndex,examsPerDay, slotNames):
    day = math.floor(slotIndex/examsPerDay)
    timePeriod = slotIndex%examsPerDay
    periodName = slotNames[timePeriod]
    timeSlot = f"Day: {day}, {periodName}"
    return timeSlot

In [4]:
def create_JSON_course_data(df, studentHeader, courseHeader):
    retData = {}
    courses = df[courseHeader].unique()

    for course in courses:
        # Num students in course
        studentsInCourse = df[df[courseHeader] == course][studentHeader].unique()
        
        if len(course) == 8:
            subj = course[:4].upper()
            if subj == "MBIO":
                subj = "BIOL"
            retData[course] = {
                "students": len(studentsInCourse),
                "conflicts": [],
                "conflictsDict": {},
                "year": int(course[4]),
                "subject": subj,
                "subjectYear": course[:5].upper()
            }
        elif len(course) == 7:
            retData[course] = {
                "students": len(studentsInCourse),
                "conflicts": [],
                "conflictsDict": {},
                "year": int(course[3]),
                "subject": course[:3].upper(),
                "subjectYear": course[:4].upper()
            }
        else:
            print("it's time to contact the developers again")
        for conflict in courses:
            if course != conflict: 
                studentsInOtherCourse = df[df[courseHeader] == conflict][studentHeader].unique()
                studentsInConflicts = set(studentsInOtherCourse) & set(studentsInCourse)
                if (studentsInConflicts):
                    retData[course]['conflicts'].append(conflict)
                    retData[course]['conflictsDict'][conflict] = len(studentsInConflicts)

        retData[course]['numConflicts'] = len(retData[course]['conflictsDict'])
    return retData

In [5]:
def dict_to_df_no_rooms(scheduleDict, courseDict, deferralRate):
    listCourses = [(key, value['courses']) for key,value in scheduleDict.items()]
    retDf = pd.DataFrame(listCourses, columns=["Time Slot", "Courses"])
    retDf = retDf.explode("Courses").reset_index(drop=True)
    
    def addExpectedDeferrals(courseName, courseDict, deferralRate):
        return math.ceil(courseDict[courseName]['students']*deferralRate)
    
    retDf['Expected Deferrals'] = retDf['Courses'].apply(addExpectedDeferrals, args=(courseDict, deferralRate))
    return retDf

# Global Variables

In [6]:
coursesJSON = create_JSON_course_data(inCsv, "PIDM", "COURSE_IDENTIFICATION")

In [7]:
sortedCourses = dict(sorted(coursesJSON.items(), key=lambda item: item[1]['students'], reverse=True))

In [8]:
rooms = {key: value for key, value in sorted(inRooms.items(), key=lambda item: item[1], reverse=True)}

# Algorithms To Test

- [Done]        Algo 1 - Graph Coloring
- Algo 2 - ??
- Algo 3 - Recursive (Check every possible combination) ??

Then Perform Analysis - Which one is more optimal: 
- least conflicts
- uses number of rooms correctly
- shortest schedule
- Quickest to run

# Graph Coloring Scheduler

### Helper Functions

In [9]:
def json_to_adj_list(courseDict, nameIndex):
    vertices = list(courseDict.keys())
    adjList = [[] for _ in vertices]

    for key, value in courseDict.items():
        vertex = nameIndex[key]
        for conflict in value['conflicts']:
            if (nameIndex.get(conflict)):
                adjList[vertex].append(nameIndex[conflict])

    return adjList

In [10]:
def greedy_colouring(adjList):
    numVertices = len(adjList)
    result = [-1]*numVertices

    result[0] = 0

    available = [False]*numVertices

    for vertex in range(1, numVertices):
        
        for i in adjList[vertex]:
            if(result[i] != -1):
                available[result[i]] = True

        timeSlot = 0
        while timeSlot < numVertices:
            if available[timeSlot] == False:
                break

            timeSlot += 1

        result[vertex] = timeSlot

        for i in adjList[vertex]:
            if result[i] != -1:
                available[result[i]] = False

    return result

In [11]:
def color_to_schedule(graphColor, indexName, courseDict, deferralRate):
    numVertices = len(graphColor)
    retDict = {}
    for vertex in range(numVertices):
        timeSlot = index_to_timeslot(graphColor[vertex], examsPerDay, slotNames)
        courseName = indexName[vertex]
        if timeSlot in retDict:
            retDict[timeSlot]['courses'].append(courseName)
            retDict[timeSlot]['numStudents'] += math.ceil(courseDict[courseName]['students']*deferralRate)
        else:
            retDict[timeSlot] = {
                'courses': [courseName],
                'numStudents': math.ceil(courseDict[courseName]['students']*deferralRate)
            }
    return retDict

### Main Function

In [12]:
def graph_coloring_schedule(courseDict, deferralRate):
    nameIndex = {name: i for i, name in enumerate(courseDict.keys())}
    indexName = {index: name for name, index in nameIndex.items()}

    courseAdjList = json_to_adj_list(courseDict, nameIndex)
    
    graphColor = greedy_colouring(courseAdjList)

    schedule = color_to_schedule(graphColor, indexName, courseDict, deferralRate)
    
    return schedule

### Running & Exporting Data

In [13]:
gcSchedule = graph_coloring_schedule(sortedCourses, deferralRate)
gcDict = dict_to_df_no_rooms(gcSchedule, coursesJSON, deferralRate)
gcDict.to_excel('../data/Outputs/Algo 1.xlsx', index=False, sheet_name='Schedule')

### Notes

Graph Coloring algorithm solves the conflict problem and gives a schedule with no conflicts. 
- Is it conflict free? Yes
- Is it the most optimal one? Probably not
- Does it work? Yes
- Is it quick? From start to finish the code runs under 1 min


# Greedy Knapsack Scheduler

In [29]:
def box_score(inCourse, selectedBox, coursesJSON):
    if inCourse not in selectedBox['conflicts']:
        # highest score 
        return 1000
    else:
        score = 0
        for course in selectedBox['scheduled']:
            if inCourse in coursesJSON[course]['conflicts']:
                courseScore = -10
                if coursesJSON[inCourse]['subjectYear'] == coursesJSON[course]['subjectYear']:
                    courseScore = courseScore - math.pow(20 - coursesJSON[inCourse]['year'], 1)
                else:
                    courseScore = courseScore - math.pow(20 - coursesJSON[inCourse]['year'] - coursesJSON[course]['year'], 1)/2
                    if coursesJSON[inCourse]['subject'] == coursesJSON[course]['subject']:
                        courseScore = courseScore - 5
                courseScore = courseScore*coursesJSON[inCourse]['conflictsDict'][course]
                score += courseScore
        return score

In [30]:
def find_random_max_box(boxScores):
    maxScore = max(boxScores.values())
    maxBoxes = [box for box, score in boxScores.items() if score == maxScore]
    return random.choice(maxBoxes)


In [31]:
def add_conflicts_in_box(courses, box):
    for course in box['scheduled']:
        conflicts = list(set(courses[course]['conflicts']) & set(box['scheduled']))
        if len(conflicts):
            box['conflictsInSchedule'][course] = conflicts
            box['numConflicts'] += len(conflicts)


In [92]:
def fill_boxes(courses, numBoxes):
    schedule1 = {f'{index_to_timeslot(i, 3, slotNames)}': {'scheduled': [], 'conflicts': [], 'conflictsInSchedule': {}, 'numConflicts': 0} for i in range(numBoxes)}

    for course_id, course_info in courses.items():
        boxScores = {box_id: box_score(course_id, box, courses) for box_id, box in schedule1.items()}
        targetBoxID = find_random_max_box(boxScores)

        schedule1[targetBoxID]['scheduled'].append(course_id)
        schedule1[targetBoxID]['conflicts'] = list(set(schedule1[targetBoxID]['conflicts']) | set(course_info['conflicts']))        
        
    for _, box in schedule1.items():
        add_conflicts_in_box(sortedCourses, box)
        del box['conflicts']

    return schedule1

In [39]:
def sum_conflicts(boxes):
    totalConflicts = 0
    for box in boxes.values():
        totalConflicts += box['numConflicts']
    return totalConflicts

In [79]:
def find_best_boxes(courseDict, numBoxes, runs=2000):
    minHeap = []

    for i in range(runs):
        boxes = fill_boxes(courseDict, numBoxes)
        totalConflicts = sum_conflicts(boxes)
        
        if len(minHeap) < 3:
            heapq.heappush(minHeap, (-totalConflicts, i, boxes)) 
        else:
            heapq.heappushpop(minHeap, (-totalConflicts, i, boxes))  

    return [heapq.heappop(minHeap)[2] for _ in range(len(minHeap))][::-1]

In [133]:
output = find_best_boxes(sortedCourses, 15)

In [142]:
def dict_to_df(scheduleDict, coursesJSON):
    listCourses = [(key, value['scheduled']) for key,value in scheduleDict.items()]
    retDf = pd.DataFrame(listCourses, columns=["Time Slot", "Courses"])
    retDf = retDf.explode("Courses").reset_index(drop=True)

    conflictsDict = {}
    for box in scheduleDict.values():
        if box['conflictsInSchedule']:
            for course, conflicts in box['conflictsInSchedule'].items():
                courseConflicts = {}
                for conflict in conflicts: 
                    if coursesJSON[course]['conflictsDict'][conflict]:
                        courseConflicts[conflict] = coursesJSON[course]['conflictsDict'][conflict]
                conflictsDict[course] = courseConflicts

    retDf['Conflict Details'] = retDf['Courses'].map(lambda x: conflictsDict.get(x, {}))
    
    return retDf

In [144]:
df = dict_to_df(output[1], sortedCourses)

In [148]:
i = 0
for schedule in output:
    df = dict_to_df(schedule, sortedCourses)
    df.to_excel(f'../data/Outputs/Algo 2 {i}.xlsx', index=False, sheet_name='Schedule')
    i +=1

In [145]:
df.to_excel('../data/Outputs/Algo 2.xlsx', index=False, sheet_name='Schedule')

In [96]:
sum_conflicts(output[2])

18

In [98]:
sortedCourses

{'STAT1000': {'students': 1482,
  'conflicts': ['BIOL1410',
   'MBIO1220',
   'MATH2090',
   'BIOL1000',
   'MATH1010',
   'BIOL2390',
   'BIOL2410',
   'MATH1500',
   'COMP3380',
   'BIOL2520',
   'CHEM1100',
   'PHYS3650',
   'COMP3020',
   'COMP3190',
   'COMP2160',
   'BIOL1010',
   'MATH1020',
   'MATH1080',
   'CHEM1018',
   'MATH1520',
   'MATH2132',
   'COMP1020',
   'BIOL1300',
   'MBIO2700',
   'BIOL3290',
   'MATH2130',
   'MATH1700',
   'BIOL1020',
   'MATH1300',
   'MATH2720',
   'PHYS2272',
   'PHYS2496',
   'PHYS2600',
   'MBIO3032',
   'BIOL3100',
   'BIOL4100',
   'BIOL2260',
   'CHEM2100',
   'BIOL2500',
   'MBIO1010',
   'ASTR1810',
   'CHEM1110',
   'CHEM2720',
   'BIOL2242',
   'CHEM3700',
   'MBIO3410',
   'CHEM1120',
   'COMP2140',
   'MATH3132',
   'COMP1010',
   'MATH1240',
   'MATH2030',
   'CHEM2600',
   'PHYS1020',
   'CHEM2122',
   'CHEM2730',
   'COMP1500',
   'ASTR2000',
   'CHEM2700',
   'MBIO3010',
   'MATH1210',
   'BIOL2200',
   'CHEM2510',
   'STAT33